In [1]:
import Pkg;
Pkg.activate(joinpath(@__DIR__, ".."));
Pkg.instantiate();
using DelimitedFiles
using CSV
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random;

  Activating environment at `~/planar_quadruped_landing/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl");

WebIO._IJuliaInit()

In [3]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Some parameters
dt = 0.005
N = 61
times = range(0, dt * (N - 1), length=N)
k_trans = 21
n = 15
m = 5

# Initial condition. Currently, we assume the initial mode ID is 1
xinit = zeros(n)
xinit[1] = -lb / 2.5                # xb
xinit[2] = sqrt(l1^2 + l2^2) + 0.1  # yb
xinit[3] = -20 * pi / 180           # theta
xinit[6] = -lb                      # x2
xinit[7] = 0.2                      # y2
xinit[9] = -1.0                     # yb_dot
xinit[12] = -1.0                    # y2_dot

init_mode = 1

# Desired final state
xterm = zeros(n)
xterm[1] = -lb / 2           # xb
xterm[2] = sqrt(l1^2 + l2^2) # yb
xterm[6] = -lb;              # x2

In [4]:
# Reference Trajectory
Xref, Uref = reference_trajectory(model, N, k_trans, xterm, init_mode, dt);

In [5]:
# Objective
Q = Diagonal([10.0; 10.0; 1.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 1.0; 10.0; 10.0; 10.0; 10.0; 0.0])
R = Diagonal(fill(1e-3, 5))
R[end, end] = 0.0
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end
push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]));

In [6]:
# Define the NLP
nlp = HybridNLP(model, obj, init_mode, k_trans, N, xinit, xterm);

In [12]:
# Initial guess
# Random.seed!(1)

# Uguess = [u + 0.1*randn(length(u)) for u in Uref]
# Xguess = [x + 0.1*randn(length(x)) for x in Xref]

# initialize Xguess
Xguess = [zeros(n) for x in Xref]
k_trans = nlp.k_trans

for k = 1:N
    if k <= k_trans
        Xguess[k] = xinit + (xterm - xinit) / (k_trans - 1) * (k - 1)
    else
        Xguess[k][1:14] = xterm[1:14]
    end
    
    Xguess[k][end] = dt * (k - 1)
end

# initialize Uguess
Uguess = [zeros(m) + rand(m) for u in Uref]
for k = 1: N-1
    Uguess[k][end] = dt
end

In [13]:
Z0 = packZ(nlp, Xguess, Uref)

Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-3)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  1252665
Number of nonzeros in inequality constraint Jacobian.:    74115
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     1215
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      121
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1031
Total number of inequality constraints...............:       61
        inequality constraints with only lower bounds:       61
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       

([-0.199999999888274, 0.45355339060596006, -0.34906585039844434, -3.801612852007844e-11, 0.0003819513794922862, -0.49999999985324683, 0.19999999999856843, 1.327730325064822e-11, -0.9999999999980587, -1.541639374487778e-12  …  -0.500000000189997, 1.8586359899774105e-13, 4.467343727772998e-13, 1.322663959179018e-13, 2.0178732052830547e-12, 2.2011020082633186e-12, 1.3024370762088127e-14, 2.933380502757403e-11, 1.3045565173069824e-14, 0.38799521421836064], Ipopt.Optimizer)

In [14]:
Z_sol[1:15] - xinit

15-element Vector{Float64}:
  1.1172601732667431e-10
  1.2686240946635507e-11
  4.2155168245017194e-13
 -3.801612852007844e-11
  0.0003819513794922862
  1.4675316517553938e-10
 -1.4315770791029081e-12
  1.327730325064822e-11
  1.9413359808595487e-12
 -1.541639374487778e-12
 -6.741695553772551e-13
  1.7997418726656278e-5
 -4.2058553455957116e-13
  3.629174996117791e-12
 -4.795615456914076e-23

In [15]:
Z_sol[end-14:end] - xterm

15-element Vector{Float64}:
 -1.1453782366999121e-11
 -1.0104528325172168e-11
 -7.844733044533612e-13
 -1.9436413933508376e-11
  1.8586995585589915e-13
 -1.89997018118504e-10
  1.8586359899774105e-13
  4.467343727772998e-13
  1.322663959179018e-13
  2.0178732052830547e-12
  2.2011020082633186e-12
  1.3024370762088127e-14
  2.933380502757403e-11
  1.3045565173069824e-14
  0.38799521421836064

In [16]:
theta = zeros(N)
for k = 1:N
    theta[k] = Z_sol[3+20*(k-1)]
end
@show theta;

theta = [-0.34906585039844434, -0.3489565084652224, -0.34863586785943484, -0.3481256380523912, -0.34744959707250467, -0.3466251020178286, -0.3456675507615372, -0.3445916973791306, -0.3434118186195406, -0.3421416120271847, -0.34079414920318274, -0.33938186964655326, -0.33791642117064713, -0.33640824396977415, -0.33486617431777677, -0.33329760152956633, -0.3317088602879703, -0.3301032071807728, -0.32850516330701995, -0.32691198741084865, -0.31600750863461374, -0.308587240447929, -0.30110394471490837, -0.2935384047205221, -0.2858068760382454, -0.2779841862013507, -0.27022424784541416, -0.26259338306902485, -0.2550726511992168, -0.24760722531782592, -0.24014599047996238, -0.2326788656815055, -0.22519959712343715, -0.21762859541903457, -0.2098559857024605, -0.2018130581017257, -0.19347311784718735, -0.18483945922393213, -0.1759408372362745, -0.16683104563372708, -0.15757454196440937, -0.14821770774774076, -0.138780653153999, -0.1292788947849903, -0.11974289194419657, -0.11022015859616903, -

In [17]:
dt = zeros(N-1)
for k = 1:N-1
    @show dt[k] = Z_sol[20 + 20 * (k-1)]
end

dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.008999989154076177
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.008999987671933093
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.00899998356156425
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.00899997972286566

In [18]:
# display(Z_sol)
writedlm("data_optimal_solution_found_1.csv", Z_sol, ',')